In [1]:
#installed with pip
from pandas import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error import HTTPError
from collections import namedtuple

# built-in libraries
import datetime
import re
import sqlite3
import urllib
import csv
import datetime

In [2]:
# Read state summary report for District into Pandas df; filter for needed fields and assign headers
insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", 
                   usecols=[2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 17, 18, 80, 81], encoding="ISO-8859-1")
insp.columns = ["county", "licnum", "sitename", "streetaddy", "cityaddy", "zip",
                "inspnum", "insptype", "inspdispos", "inspdate", "totalvio", "highvio", "licid", "visitid"]
county_sought = 'Marion' # uncomment if particular county sought
insp = insp[(insp.county == county_sought)] # uncomment if particular county sought
insp.sitename = insp.sitename.str.title() #titlecase sitename
insp.streetaddy = insp.streetaddy.str.title() #titlecase street adress
insp.cityaddy = insp.cityaddy.str.title() #titlecase city
insp = insp.applymap(lambda x: str(x).strip() if len(str(x).strip()) else None) #strip whitespace, replace empty vals
insp['visitid'] = insp['visitid'].apply(int) # so it can be filtered against df below

# Read in records from database of earlier reports, create df to filter against new reports in df above.
conn = sqlite3.connect("rinspect.sqlite")
df = pd.read_sql_query("select * from fdinsp;", conn)
df
conn.close()

unique_vals = insp[~insp.visitid.isin(df.visitid)] #filter

In [3]:
# Build a list of URLs to the inspectors detailed reports that will get scraped
result = []
result_for_urls = result # for url list
result_for_list = result.append("NULL") # later into db, space for user-input datetime

# takes LicenseID and VisitID, passes it into the urls for detailed reports later
for index, rows in unique_vals.iterrows():
    visitid = rows['visitid']
    licid = rows['licid']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp? \
        InspVisitID= %s &id= %s" % (visitid, licid)
    urls = urls.replace(' ', '')
    result.append(urls)
urlList = result
urlList.pop(0) # get rid of first "Null" from append above
len(urlList)

0

In [4]:
# Place data from state summary report df into database
# first, interate through the df to return tuples
var = list(unique_vals.itertuples(index='visitid', name=None))
# populate database table for inspection summmary reports
sqlite_file = 'rinspect.sqlite'
fdinsp_table = 'fdinsp' # table for summary data
id_field = 'visitid' # ID column
time_now = 'now' # column for user-input timestamp; until then = NULL

# connect to database and insert new summary report data
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c.executemany('''INSERT OR IGNORE INTO fdinsp (librow, county, licnum, sitename,
              streetaddy, cityaddy, zip, inspnum, insptype, inspdispos,
              inspdate, totalvio, highvio, licid, visitid)
              VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', var)
conn.commit()
conn.close()

In [5]:
urlList = ['https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6632858&id=6605529', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6653464&id=4070634', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6602877&id=4070634', 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6577708&id=2129302',]# a shorter try for a list
url = 'https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6632858&id=6605529'

In [6]:
# This gets observations details from each deetailed report
obs = {} # dict of observations for all reports
report = {} # dictionary for each report findings

def make_obs():
    global report
    global obs
    visitid = url.split("VisitID=")[1].split("&")[0]
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html.read(), 'lxml')
    details = soup.findAll('font', {'face': 'verdana'})[42:]
    details = str(details)
    details = re.sub(r"\<.*\>", "", details) # regex string matches for tags in the soup
    details = re.sub(r"\t", "", details) # remove literal strings
    details = re.sub(r"\n", "", details)
    details = re.sub(r"\r", "", details)

    report[visitid] = details  
    #obs.append(report)
    
    obs.update( {visitid : details} )

for url in urlList:
    make_obs()
    
# This is working pretty well now but what it is? A dictionary with values as strings or tuples?

# MORE THOUGHTS:
# If we are just going to take this list, it still needs to be cleaned up and formatted, and
# more stuff added from summary report details.
# If we are going to put values into database, observations should be cleaned up and separated.
# We'd then need another script to pull values from database to write report for publication.

In [7]:
print(obs) # just to see what it looks like.

{'6632858': '[Basic - Carbon dioxide/helium tanks not adequately secured. Observed 2 compressed gas tanks on the wait line, not secured. The chain was added. **Corrected On-Site**Basic - Case/container/bag of food stored on floor in dry storage area. Observed bulk oil stored on the floor in dry storage.Basic - In-use utensil stored in standing water less than 135 degrees Fahrenheit. Observed rice spoon in 84° water on the cook line. The water was discarded. **Corrected On-Site* ALSO, tongs were on equipment handles, protruding into the cook line walkway. The tongs were moved. **Corrected On-Site**Basic - In-use wiping cloth/towel used under cutting board. Observed cloth under cutting board. The towel was removed. **Corrected On-Site**Basic - Soiled cloths, linen, aprons, coats, or other uniform apparel not kept in a suitable container prior to laundering. Observed soiled apron stored on dry good shelves, over prep area. The apron was placed in a soiled linen bag.Basic - Walk-in cooler 